<a href="https://colab.research.google.com/github/gabrielbruschi/predicao_acoes/blob/main/pipeline-lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 3.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler #normalizar os dados
import matplotlib.pyplot as plt #grafico
from sklearn.model_selection import train_test_split #treinamento e teste

In [17]:
plt.style.use('ggplot')

In [5]:
import yfinance as yf

In [19]:
obj = yf.Ticker('petr4.sa')

In [20]:
data = obj.history(start='2002-01-01', end='2022-05-01')
df = data.dropna()
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-01-01,2.328508,2.328508,2.328508,2.328508,0,0.0,0.0
2002-01-02,2.348994,2.363562,2.328509,2.333061,23920000,0.0,0.0
2002-01-03,2.362196,2.367204,2.339890,2.348994,39043200,0.0,0.0
2002-01-04,2.321680,2.348994,2.303471,2.334427,21500800,0.0,0.0
2002-01-07,2.321680,2.403622,2.321680,2.385412,40406400,0.0,0.0
...,...,...,...,...,...,...,...
2022-04-25,21.899458,22.242894,21.585253,22.001759,66413900,0.0,0.0
2022-04-26,22.001760,22.345194,21.841002,21.965223,48764100,0.0,0.0
2022-04-27,22.191745,22.330580,21.884846,21.965223,41881300,0.0,0.0


In [22]:
df_acao_fec = df[['Close']]
df_acao_fec

In [24]:
#verificar a quantidade de linhas
qtd_linhas = len(df_acao_fec)

qtd_linhas_treino = round(.70 * qtd_linhas)
qtd_linhas_teste = qtd_linhas - qtd_linhas_treino

info = (
    f"linhas treino = 0{qtd_linhas_treino}"
    f"linhas teste = {qtd_linhas_treino}:{qtd_linhas_treino + qtd_linhas_teste}"
)

info

'linhas treino = 03562linhas teste = 3562:5088'

In [25]:
#Normalizar os dados
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_acao_fec)

In [26]:
train = df_scaled[:qtd_linhas_treino]
test = df_scaled[qtd_linhas_treino: qtd_linhas_treino + qtd_linhas_teste] 

print( len(train), len(test))

3562 1526


In [27]:
#Convert an array of values into a df matrix
#Rede LSTM
def create_df(df, steps=1):
  dataX, dataY = [], []
  for i in range(len(df) - steps - 1):
    a = df[i:(i+steps), 0]
    dataX.append(a)
    dataY.append(df[i + steps, 0])
  return np.array(dataX), np.array(dataY)

In [28]:
#Gerando dados de treino e teste
steps = 15 
X_train, Y_train = create_df(train, steps)
X_test, Y_test = create_df(test, steps)

In [29]:
# Gerando os dados que o modelo espera
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1) #1 'e a qntd de features
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [30]:
#Montando a rede
model = Sequential()
model.add(LSTM(35, return_sequences=True, input_shape=(steps, 1))) #35 neuronios, o return_seq = true : pega a informacao q sai e reinsere, dados com memoria
model.add(LSTM(35, return_sequences=True))
model.add(LSTM(35))
model.add(Dropout(0.05)) #ANTES ERA 0.2 #nao causar um overfit (rede muito treinada), dou uma penalizada na feature
model.add(Dense(1)) #saida unica do preco que queremos prever

In [31]:
model.compile(optimizer='adam', loss='mse') #adam mais usado, loss: minimun sequer error, ver o quanto minha rede esta performando
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 35)            5180      
                                                                 
 lstm_1 (LSTM)               (None, 15, 35)            9940      
                                                                 
 lstm_2 (LSTM)               (None, 35)                9940      
                                                                 
 dropout (Dropout)           (None, 35)                0         
                                                                 
 dense (Dense)               (None, 1)                 36        
                                                                 
Total params: 25,096
Trainable params: 25,096
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Treinamento do modelo
validation = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=200, batch_size=15, verbose=2)
# ele vai validando para ver como esta perfomando, batch_size 'e quantidade de dias pra tras, verbose pra ver como as informacoes estao sendo plotadas

In [ ]:
#plt.figure(figsize=(16,8))
plt.plot(validation.history['loss'], label='Training loss')
plt.plot(validation.history['val_loss'], label='Validation loss')
plt.legend()
plt.title('Validação e Performace do algoritmo LSTM')

In [ ]:
#Fazendo previsao

prev = model.predict(X_test)
prev = scaler.inverse_transform(prev) #removo a normalizacao
prev # precos previstos

In [ ]:
#previsão para os proximos 10 dias

lenght_test = len(test)
lenght_test

In [ ]:
#pegar os ultimos dias que são o tamanho do meu step - 15 dias

days_input_steps = lenght_test - steps
days_input_steps

In [ ]:
#transforma em array

input_steps = test[days_input_steps:]
input_steps = np.array(input_steps).reshape(1,-1)
input_steps # informacao usada para prever os 10 dias a frente
#utilizamos os 15 dias para trás para prever os proximos 10 dias a frente

In [ ]:
#transformar em lista

list_output_steps = list(input_steps)
list_output_steps = list_output_steps[0].tolist()
list_output_steps

In [ ]:
#loop para prever os proximos 10 dias

pred_output = []
i = 0
n_future = 10

while (i < n_future):
    
    if (len(list_output_steps) > steps):
        input_steps = np.array(list_output_steps[1:])
        print("{} dia. Valores de entrada -> {}".format(i,input_steps))
        input_steps = input_steps.reshape(1,-1)
        input_steps = input_steps.reshape((1, steps, 1))
        #print(input_steps)
        pred = model.predict(input_steps, verbose = 0)
        print("{} dia. Valor previsto -> {}".format(i, pred))
        list_output_steps.extend(pred[0].tolist())
        list_output_steps = list_output_steps[1:]
        #print(list_output_steps)
        pred_output.extend(pred.tolist())
        i = i + 1
    else:
        input_steps = input_steps.reshape((1, steps, 1))
        pred = model.predict(input_steps, verbose = 0)
        print(pred[0])
        list_output_steps.extend(pred[0].tolist())
        print(len(list_output_steps))
        pred_output.extend(pred.tolist())
        i = i + 1

print(pred_output)

In [ ]:
#transformar a saida

prev = scaler.inverse_transform(pred_output)
prev = np.array(prev).reshape(1, -1)
list_output_prev = list(prev)
list_output_prev = prev[0].tolist()
list_output_prev

In [ ]:
#pegar as datas de previsão 

dates = pd.to_datetime(df['data_pregao'])
predict_dates = pd.date_range(list(dates)[-1] + pd.DateOffset(1), periods = 10, freq='b').tolist() #freq b = dias bussins
predict_dates 

In [ ]:
#cria dataframe de previsao

forecast_dates = []
for i in predict_dates:
    forecast_dates.append(i.date())
    
df_forecast = pd.DataFrame({'data_pregao': np.array(forecast_dates), 'preco_fechamento': list_output_prev})
df_forecast['data_pregao'] = pd.to_datetime(df_forecast['data_pregao'])

df_forecast = df_forecast.set_index(pd.DatetimeIndex(df_forecast['data_pregao'].values))
df_forecast = df_forecast[df_forecast['data_pregao'] > '2022-04-01']
df_forecast.drop('data_pregao', axis = 1, inplace = True)
df_forecast

In [ ]:
df_acao_fec = df[['data_pregao', 'preco_fechamento']]
df_acao_fec = df_acao_fec.set_index(pd.DatetimeIndex(df_acao_fec['data_pregao'].values))
df_acao_fec = df_acao_fec[df_acao_fec['data_pregao'] > '2022-01-01']
df_acao_fec.drop('data_pregao', axis=1, inplace=True)
df_acao_fec

In [ ]:
#plotar o grafico

#plt.figure(figsize=(16,8))
plt.plot(df_acao_fec['preco_fechamento'], label='Preço de Fechamento')
plt.plot(df_forecast['preco_fechamento'], label='Preço Previsto')
plt.legend()
plt.title('Previsão de 10 dias da ação PETR4.SA')
